In [1]:
import json
import time
import base64
from algosdk.v2client import algod
from algosdk import mnemonic, account
from algosdk import transaction

algod_token =<algo-token>
algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
purestake_token = {'X-Api-key': algod_token}

In [2]:
private_key1, address1 = account.generate_account()
private_key2, address2 = account.generate_account()

In [10]:
address1

'ZHFDAUSZXTFPFJ4QPODDU6V3OEEKPRKSXR7I4D6OQGNZ4QCSDLHM6Q5LJI'

In [3]:
algodclient = algod.AlgodClient(algod_token, algod_address, headers=purestake_token)

In [12]:
account_info = algodclient.account_info(address1)
print("Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")

Account balance: 10000000 microAlgos



In [9]:
account_info = algodclient.account_info(address2)
print("Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")

Account balance: 0 microAlgos



In [5]:
params = algodclient.suggested_params()

gh = params.gh
first_valid_round = params.first
last_valid_round = params.last
fee = params.min_fee
send_amount = 1000000

existing_account = address1
send_to_address = address2

In [6]:
tx = transaction.PaymentTxn(existing_account, fee, first_valid_round, last_valid_round, gh, send_to_address, send_amount, flat_fee=True)
signed_tx = tx.sign(private_key1)

In [7]:
# Function from Algorand Inc. - utility for waiting on a transaction confirmation
def wait_for_confirmation(client, txid):
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print('Waiting for confirmation')
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print('Transaction confirmed in round', txinfo.get('confirmed-round'))
    return txinfo

In [13]:
try:
    tx_confirm = algodclient.send_transaction(signed_tx)
    print('Transaction sent with ID', signed_tx.transaction.get_txid())
    wait_for_confirmation(algodclient, txid=signed_tx.transaction.get_txid())
except Exception as e:
    print(e)

Transaction sent with ID OMF5GV5NJWGGQ45IPMM536YP3CDCPI3WTKX33NDEPCGK7SVCUYDQ
Waiting for confirmation
Transaction confirmed in round 24399576
